Here we add a dropout probability for tensorflow.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

def array_to_one_hot(lable):
    # input lable is list
    new_lable =  []
    lables =  sorted(set(lable))    
    for i in lable:
        single_lable = [0] * len(lables)
        index = lables.index(i)
        single_lable[index] = 1
        new_lable.append(single_lable)
    return np.array(new_lable)

def fake_label_to_real_label_array(fake_label, labels):
    real_label_array = []
    for i in fake_label:
        real_label_array.append(labels[i])
    return np.array(real_label_array)

def one_hot_to_array(labels):
    new_label = []
    for label in labels:
        index = label.tolist().index(1)
        new_label.append(index)
    return new_label

train = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")

feature_name = train.columns[1:] 
label_name = train.columns[0] #put the class label on the first column

train_feature = train[feature_name].as_matrix()
test_feature = test[feature_name].as_matrix()

train_label_one_hot = array_to_one_hot(train[label_name].tolist()) #tensorflow must use one-hot form
test_label_one_hot = array_to_one_hot(test[label_name].tolist())

train_label_array = np.array(train[label_name])
test_label_array = np.array(test[label_name])

labels = sorted(set(train[label_name].tolist()))

In [4]:
print(train_feature.shape)
print(train_label_one_hot.shape)

(10000, 784)
(10000, 10)


In [16]:
## compelet of preparing the data
learning_rate = 0.005
training_period = 20
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 400 # 1st layer number of features
n_hidden_2 = 200 # 2nd layer number of features
n_input = train_feature.shape[1]
n_classes = train_label_one_hot.shape[1]

In [6]:
# difine keep probability
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob=tf.placeholder(tf.float32)

In [8]:
# define variable
w1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev=1))
w2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev=0.1))
w3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev=0.1))

b1 = tf.Variable(tf.zeros([n_hidden_1])+0.1)
b2 = tf.Variable(tf.zeros([n_hidden_2])+0.1)
b3 = tf.Variable(tf.zeros([n_classes])+0.1)

In [11]:
# add layer with dropout probability
layer_1 = tf.add(tf.matmul(x, w1), b1)
layer_1 = tf.nn.relu(layer_1)
layer_1_drop = tf.nn.dropout(layer_1,keep_prob)
# Hidden layer with RELU activation
layer_2 = tf.add(tf.matmul(layer_1_drop, w2), b2)
layer_2 = tf.nn.relu(layer_2)
layer_2_drop = tf.nn.dropout(layer_2,keep_prob)
# Output layer with linear activation
pred = tf.add(tf.matmul(layer_2_drop, w3), b3)

In [12]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

# defined next_batch function
def next_batch(round_num,sample_num, data):
    # Return a total of `num` samples from the array `data`. 
    return data[sample_num*round_num:sample_num*(round_num+1)]

In [17]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for k in range(training_period):
        total_batch = int(train_feature.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = next_batch(i,batch_size,train_feature)
            batch_y = next_batch(i,batch_size,train_label_one_hot)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:0.9})
        # Display logs per epoch step
        if k % display_step == 0:
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            
            test_accuracy = accuracy.eval({x: test_feature, y: test_label_one_hot,keep_prob:1})
            train_accuarcy = accuracy.eval({x: train_feature, y: train_label_one_hot,keep_prob:1})
            print("period %d" %k,"trian:", train_accuarcy, "test:" ,test_accuracy)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # get accuarcy
    test_accuracy = accuracy.eval({x: test_feature, y: test_label_one_hot,keep_prob:1})
    train_accuarcy = accuracy.eval({x: train_feature, y: train_label_one_hot,keep_prob:1})
    
    test_acc = sess.run(accuracy,feed_dict={x: test_feature, y: test_label_one_hot,keep_prob:1.0})
    train_acc = sess.run(accuracy,feed_dict={x: test_feature, y: test_label_one_hot,keep_prob:1.0})
    # get probability matrix
    soft_y = tf.nn.softmax(pred)
    test_pred_prob_matrix = sess.run(soft_y,feed_dict={x: test_feature,keep_prob:1.0})
    train_pred_prob_matrix = sess.run(soft_y,feed_dict={x: train_feature,keep_prob:1.0})
    
    # get predicted index
    test_pred_index = sess.run(tf.argmax(pred, 1), feed_dict={x: test_feature,keep_prob:1.0})
    train_pred_index = sess.run(tf.argmax(pred, 1), feed_dict={x: train_feature,keep_prob:1.0})
    
    # get predicted label
    test_pred_label_array = fake_label_to_real_label_array(np.array(test_pred_index.tolist()), labels) #list form
    train_pred_label_array = fake_label_to_real_label_array(np.array(train_pred_index.tolist()), labels) #list form 

period 0 trian: 0.2055 test: 0.2115
period 1 trian: 0.2682 test: 0.272
period 2 trian: 0.3175 test: 0.3125
period 3 trian: 0.4258 test: 0.424
period 4 trian: 0.6125 test: 0.6065
period 5 trian: 0.5829 test: 0.579
period 6 trian: 0.7581 test: 0.751
period 7 trian: 0.8018 test: 0.803
period 8 trian: 0.8583 test: 0.858
period 9 trian: 0.9144 test: 0.915
period 10 trian: 0.9448 test: 0.95
period 11 trian: 0.9629 test: 0.9615
period 12 trian: 0.976 test: 0.9765
period 13 trian: 0.9826 test: 0.982
period 14 trian: 0.9862 test: 0.986
period 15 trian: 0.99 test: 0.989
period 16 trian: 0.9935 test: 0.9925
period 17 trian: 0.9947 test: 0.9945
period 18 trian: 0.9962 test: 0.9955
period 19 trian: 0.997 test: 0.9965


In [19]:
# here we use two ways to get train and test accuarcy
print("train_accuarcy:",train_accuarcy)
print("train_acc:",train_acc)

print("test_accuracy:",test_accuracy)
print("test_acc",test_acc)

train_accuarcy: 0.997
train_acc: 0.9965
test_accuracy: 0.9965
test_acc 0.9965


In [20]:
print("test_pred_prob_matrix:",test_pred_prob_matrix)

test_pred_prob_matrix: [[  7.69217372e-01   2.90796655e-04   2.11394921e-01 ...,   7.71797216e-03
    8.86517402e-04   9.85388062e-04]
 [  1.00000000e+00   1.45292780e-23   8.31150285e-17 ...,   9.04535098e-31
    6.10369850e-38   8.30425613e-37]
 [  1.00000000e+00   5.16882177e-34   2.98243372e-29 ...,   2.06480723e-37
    8.74148305e-29   0.00000000e+00]
 ..., 
 [  5.53881364e-06   1.90445206e-08   1.42951446e-08 ...,   1.80008810e-05
    5.72615058e-08   9.99972343e-01]
 [  1.98734227e-07   2.79165691e-09   6.09270279e-09 ...,   2.30011005e-06
    2.80512324e-09   9.99996066e-01]
 [  1.24967059e-07   1.06815690e-09   1.41931314e-10 ...,   3.17005693e-08
    3.09972337e-09   9.99994636e-01]]
